### Fine-tuning to Follow Instructions

Pretraining an LLM involves a training procedure where it learns to generate one word at a time. The resulting pretrained LLM is capable of text completion, meaning it can finish sentences or write text paragraphs given a fragment of input. However, they struggle with specific instructions, such as "fix the grammar in this text". Here, we will focus on improving the LLM's ability to follow such instructions and generate a desired response. 

In contrast to the classification fine-tuned model, this type of model can typically undertake a broader range of tasks. The former is highly specialised and easier to develop, compared to the latter which is a generalist model that works well across various tasks. 

#### Preparing a dataset for supervised instruction fine-tuning

We will download and format the instruction dataset. The dataset consists of 1,100 <i>instruction-response pairs</i>. 

In [1]:
import json
import os
import urllib
import urllib.request

def download_and_load_file(file_path, url):
    if not os.path.exists(file_path):
        with urllib.request.urlopen(url) as response:
            text_data = response.read().decode("utf-8")
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(text_data)
    else:
        with open(file_path, "r", encoding="utf-8") as file:
            text_data = file.read()
    with open(file_path, "r") as file:
        data = json.load(file)
    return data

file_path = "instruction-data.json"
url = (
    "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch"
    "/main/ch07/01_main-chapter-code/instruction-data.json"
)

data = download_and_load_file(file_path, url)
print("Number of entries:", len(data))

Number of entries: 1100


In [3]:
print("Example entry:\n", data[50])

Example entry:
 {'instruction': 'Identify the correct spelling of the following word.', 'input': 'Ocassion', 'output': "The correct spelling is 'Occasion.'"}


The entries are Python dictionary objects, consisting of 3 keys: 'instruction', 'input', and 'output'.

In [4]:
print("Another example entry:\n", data[999])

Another example entry:
 {'instruction': "What is an antonym of 'complicated'?", 'input': '', 'output': "An antonym of 'complicated' is 'simple'."}


Instruction fine-tuning involves training a model on a dataset where the input-output pairs, like those extracted above, are explicitly provided. There are various methods to format these entries for LLMs, known as <i>prompt styles</i>. A popular one is the Alpaca prompt style, which we shall use. Below, we define a format_input function that we can use to convert the entries in the data list into the Alpaca-style input format.

In [5]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )
    input_text = (
        f"\n\n### Input:\n{entry['input']}" if entry['input'] else ""
    )
    return instruction_text + input_text

In [6]:
model_input = format_input(data[50])
desired_response = f"\n\n### Response:\n{data[50]['output']}"
print(model_input + desired_response)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Identify the correct spelling of the following word.

### Input:
Ocassion

### Response:
The correct spelling is 'Occasion.'


Note that the function skips the optional input section field if it is empty. 

In [8]:
model_input = format_input(data[999])
desired_response = f"\n\n### Response:\n{data[999]['output']}"
print(model_input + desired_response)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
What is an antonym of 'complicated'?

### Response:
An antonym of 'complicated' is 'simple'.


In [9]:
# Divide the dataset into train, validation and test splits
train_portion = int(len(data) * 0.85)
test_portion = int(len(data) * 0.1)
val_portion = len(data) - train_portion - test_portion

train_data = data[:train_portion]
test_data = data[train_portion:train_portion + test_portion]
val_data = data[train_portion + test_portion:]

print("Training set length:", len(train_data))
print("Validation set length:", len(val_data))
print("Test set length:", len(test_data))

Training set length: 935
Validation set length: 55
Test set length: 110


#### Organising data into training batches

Previously, the training batches were created automatically by the PyTorch dataloader class, which employs a default <i>collate</i> function to combine lists of samples into batches. This function is responsible for taking a list of individual data samples and merging them into a single batch that can be processed efficiently by the model during training. 

However, the batching process for instruction fine-tuning is more involved and requires us to create our own custom collate function that we can plug into the DataLoader. 